### 查看GPU

上傳train.csv資料集

In [5]:
!nvidia-smi #看K80調整best size

Fri May 20 03:26:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 安裝需要的BERT套件

In [6]:
!pip install transformers -U
!pip install nlp -U
!pip install torch -U

### 載入套件

In [7]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments #BertTokenizerFast把辭拆成Token TrainingArguments成為一個class
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd #csv檔都用pandas
from sklearn.metrics import accuracy_score, precision_recall_fscore_support #sklearn.metrics模型準確度

### 讀取訓練資料集

In [8]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df)


In [9]:
df.sample(10) #看0還1 0是OK 1是不行

,ID,Name,Description,label
771,771,艾多美-益生菌,含有維持消化道健康的13種乳酸菌，可增加人體消化道好菌，減少壞菌滋生，促進消化道蠕動使排便順暢,0
840,840,康采檸檬酸鈣粉-avon,最高320毫克德國大廠檸檬酸鈣一天2包相當於6.4杯牛奶鈣含量，輕鬆補足每天所需。奶素可食,0
160,160,宏醫雙效抗氧化健康咖啡組,「陳湘青口述：『...酸就是開始肥胖跟癌症...平均值一個禮拜幾乎可以到4-5公斤...藥局...,1
728,728,美力人蔘(美樂家),美力人蔘精神元氣足?專為忙碌的生活型態所量身設計，含高麗蔘抽出液、葡萄籽萃取物、牛磺酸及多重...,0
417,417,金鼎研究100%天然玉茄胡素-木鱉果（純素食品）,「100%天然玉茄胡素…保護眼睛的營養品」「鉀可以協助心臟功能及預防血壓升高…低鈉高鉀…可以...,1
117,117,老薑桂圓紅棗茶磚,「還能促進膽汁合成速率...(紅茶)...有利尿功能...(黑糖)止疼、行血、活血散寒......,1
293,293,御芝靈-靈芝茶,「免疫調節、耐缺氧、抗疲勞…延緩衰老功能…防癌…具有保護肝臟、降血壓血糖…抗自由基，預防各種...,1
897,897,https://tw.buy.yahoo.com/gdsale/GNC健安喜-DHA魚油60...,每顆含有600毫克的DHA。-5倍純化技術，不含重金屬汞、鎘、鉛和多氯聯苯。-本配方為容易吞...,0
411,411,順天本草長大人成長套組(女)x6送【五珍補益醇菁x4+籃球x1】,「個子小，總坐前排」,1
101,101,FASTANA蛋白窈窕輕飲,「喝蛋白瘦身…2個月瘦18kg…FASTANA窈窕輕飲…MT小姐31歲…體重89.1kg，體...,1


### 載入原始模型

In [10]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext') #把預訓練模型載入 哈工大的chinese bert https://huggingface.co/

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

###  載入分詞模型

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

### 設定執行參數

In [12]:
RANDOM_SEED = 5
MAX_LEN = 512 #模型多大token多大 32 64 128隨便調
EPOCHS=5
BATCH_SIZE = 8 #神經網路記憶體

### 把資料集分成訓練、測試、驗證

In [13]:
shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 取一筆出來瞧瞧(已經打亂)

In [14]:
train_dataset[2]

{'Description': '「美國藥物管理局頒發認證去醣基大豆異黃酮，分子量小…具有以下多重功效：更年期產生的憂鬱、暴躁易怒改善…失眠、頭暈、潮紅燥熱、耳鳴改善…調節賀爾蒙平衡再生…解決皮膚老化皺紋回春…改善水腫、脂肪堆積恢復曲線」「(口述)因為台灣人大腸癌是第一名…通常台灣發現都已竟是第四期末期了，像豬哥亮、葉教授…(口述)薛忠哥吃完連息肉都改善了，降低你的癌變…宿便堆積的問題…(口述)雪芳…醫生判斷原本腸道年齡58歲，現在是25歲…大腸癌每年新發生病例已突破1.5萬人，發生人數連續9年居冠」',
 'ID': 387,
 'Name': '長庚團隊研發35+異黃酮豐潤組',
 'label': 1}

In [15]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

### 定義分詞器

In [16]:
def tokenize(batch):
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True) #max_length=MAX_LEN每句話只取512字 多餘的刪掉 不夠的補字

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 定義評測準則，訓練器

In [17]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

### 開始訓練

In [18]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 749
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 470


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 94
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.9893617021276596,
 'eval_f1': 0.9915966386554621,
 'eval_loss': 0.0698925107717514,
 'eval_precision': 1.0,
 'eval_recall': 0.9833333333333333,
 'eval_runtime': 3.409,
 'eval_samples_per_second': 27.574,
 'eval_steps_per_second': 3.52}

### 將模型存檔

In [19]:
model.save_pretrained("bert_food_ad")

Configuration saved in bert_food_ad/config.json
Model weights saved in bert_food_ad/pytorch_model.bin


### 用TENSORBOARD查看訓練結果

In [21]:
# %reload_ext tensorboard
%tensorboard --logdir='./logs' # --host 192.168.1.113
#from tensorboard import notebook
#notebook.list()
#notebook.display(port=6006, height=1000)

UsageError: Line magic function `%tensorboard` not found.


### 載入已訓練好的模型

In [22]:
from transformers import BertConfig

test_config = BertConfig.from_json_file('./bert_food_ad/config.json')
test_model = BertForSequenceClassification.from_pretrained('./bert_food_ad/pytorch_model.bin', config=test_config)
model.eval()

loading weights file ./bert_food_ad/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at ./bert_food_ad/pytorch_model.bin.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [23]:

print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


### 在此輸入測試文字

In [24]:
test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'

MAX_LEN = 512

encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

In [25]:
encoded_review

{'input_ids': tensor([[ 101, 1872, 2485, 1048, 4554, 1213,  119,  119,  119, 6362, 4706, 3209,
         4680,  119,  119,  119, 2834, 3709, 1265,  119,  119,  119,  924, 5498,
          119,  119,  119, 4649, 5604, 8038,  119,  119,  119,  924, 6362, 4649,
         5604, 8024, 3938, 5227, 4649, 2244, 1558, 7539, 8024, 7360,  856, 3130,
         2697,  119,  119,  119, 3688, 7773,  100,  100,  914, 6868, 3688, 7773,
          778, 7927,  119,  119,  119, 3867, 1265, 5143, 5186,  520,  519, 2485,
         1265, 1048, 4554, 5143, 5186, 8024, 3300, 1221, 6310, 3146, 6882, 3130,
         7768, 6549,  119,  119,  119, 3926, 7370, 5632, 4507, 1825, 2454, 5227,
         5439, 1265,  119,  119,  119, 6310, 3146, 5591, 5517, 6887, 3582, 5543,
          119,  119,  119, 6310, 4415, 4649, 5604, 1558, 7539,  119,  119,  119,
         6330, 7927, 3362, 1419, 3300, 6258, 1914, 2485, 1920, 4638, 2834, 3709,
         1265, 1212,  520,  519, 7349, 3632, 2772, 3938, 2208, 4567, 4578, 4638,
         4634,

### 進行測試

In [26]:
test_model = test_model.to('cuda')

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

classnames = ['Passed', 'Failed']

print(f'Review text: {test_text}')
print(f'Result  : {classnames[result]}')

Review text: 增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常
Result  : Failed


In [27]:
!nvidia-smi

Fri May 20 03:49:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |   8372MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------